# Caffe2 Introduction

References
* https://caffe2.ai/docs/intro-tutorial.html

In [1]:
from caffe2.python import workspace, model_helper
import numpy as np

## Blobs and Workspace, Tensors

In [2]:
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.23807916 0.35985102]
  [0.91121706 0.47594085]
  [0.58230481 0.38613892]]

 [[0.36846601 0.9202981 ]
  [0.86807489 0.83688508]
  [0.88654783 0.41495028]]

 [[0.77081172 0.57133557]
  [0.2217813  0.84453718]
  [0.22199334 0.3167732 ]]

 [[0.61116601 0.87823169]
  [0.22383377 0.64640645]
  [0.08575468 0.70428389]]]
(4, 3, 2)
[[[0.23807916 0.35985102]
  [0.91121706 0.47594085]
  [0.58230481 0.38613892]]

 [[0.36846601 0.9202981 ]
  [0.86807489 0.83688508]
  [0.88654783 0.41495028]]

 [[0.77081172 0.57133557]
  [0.2217813  0.84453718]
  [0.22199334 0.3167732 ]]

 [[0.61116601 0.87823169]
  [0.22383377 0.64640645]
  [0.08575468 0.70428389]]]


## Nets and Operators

### 1. Model defintion

In [3]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

True

In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")

In [5]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

In [6]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



### 2. Executing

In [7]:
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net)

True

In [8]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [9]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.08679973 0.12714049 0.09987643 0.0974431  0.0891405  0.10393751
  0.09626511 0.10909824 0.09267303 0.09762583]
 [0.08830051 0.1183781  0.09979241 0.09179076 0.09414592 0.10906307
  0.09054583 0.10960647 0.09924439 0.09913248]
 [0.08792613 0.1102309  0.09459518 0.09859088 0.08616371 0.10677432
  0.10494146 0.11025261 0.10360478 0.09692001]
 [0.0940778  0.0966908  0.08952153 0.10259698 0.09149273 0.10193258
  0.09115592 0.11856576 0.09422557 0.11974041]
 [0.08379909 0.10912321 0.10611124 0.10054064 0.0985202  0.10488749
  0.09661508 0.10464136 0.09981644 0.09594522]
 [0.09532934 0.13500457 0.09487301 0.08920389 0.09108158 0.10568178
  0.10352152 0.10006067 0.08803672 0.09720695]
 [0.08306641 0.10981201 0.10356744 0.09534607 0.09189891 0.10536124
  0.09457405 0.11271288 0.10280445 0.10085656]
 [0.08405121 0.11464258 0.09878629 0.0953908  0.09277253 0.10356691
  0.09900262 0.11397517 0.10547944 0.09233259]
 [0.09188846 0.1074255  0.10188085 0.09660637 0.09033471 0.09856199
  0.09383885

### 3. Backward pass

In [10]:
m.AddGradientOperators([loss])
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net, overwrite=True)

True

In [11]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label

In [12]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [13]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.09632629 0.08305414 0.09914054 0.11698696 0.09126918 0.11371178
  0.10383841 0.08373358 0.10532628 0.10661281]
 [0.09943102 0.09407094 0.0908573  0.09413522 0.0923399  0.10849854
  0.11732399 0.08344328 0.10256005 0.11733968]
 [0.10604601 0.08872339 0.09771119 0.11058455 0.0966     0.11265538
  0.1010702  0.09310079 0.09670738 0.09680113]
 [0.09845086 0.09524488 0.10663462 0.10194623 0.08531986 0.12209049
  0.10183505 0.08579805 0.10082951 0.10185042]
 [0.09170969 0.08624147 0.10199437 0.105795   0.09557021 0.11909096
  0.11504228 0.09188154 0.09665626 0.09601833]
 [0.09194015 0.09486487 0.10334487 0.1002294  0.09093742 0.11864112
  0.11984234 0.07660149 0.10222556 0.10137256]
 [0.10240609 0.09426099 0.09805542 0.11012942 0.08993021 0.11281668
  0.10019438 0.08638358 0.10510419 0.10071895]
 [0.10740739 0.0944377  0.09618554 0.10625327 0.08974446 0.12955724
  0.10086174 0.08852451 0.0948196  0.09220871]
 [0.09787394 0.07968421 0.10419039 0.10200994 0.10070807 0.11440475
  0.10484736